In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


In [38]:
# look shiny data
data = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")
#data = data.loc[:1000, :]
data = data.drop("id", axis=1)
data

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,e,8.80,f,s,u,f,a,c,w,4.51,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,p,4.51,x,h,o,f,a,c,n,4.79,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,e,6.94,f,s,b,f,x,c,w,6.85,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,e,3.88,f,y,g,f,s,NaN,g,4.16,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,e,5.85,x,l,w,f,d,NaN,w,3.37,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,12.14,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,e,10.88,s,NaN,w,t,d,c,p,6.65,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,p,7.82,x,e,e,f,a,NaN,w,9.51,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,e,9.45,p,i,n,t,e,NaN,p,9.13,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [39]:
print(data['class'].unique())

['e' 'p']


In [40]:
data.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [41]:
# find missing data
data.isnull().sum()

class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [42]:
# replacing missing numerical values with mean
data[data["cap-diameter"].isna()].replace('na', np.nan, inplace=True)
data[data["cap-diameter"].isna()] = np.mean(data["cap-diameter"])
data.isnull().sum()

/tmp/ipykernel_33/2724737852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data["cap-diameter"].isna()].replace('na', np.nan, inplace=True)


class                         0
cap-diameter                  0
cap-shape                    40
cap-surface              671020
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258433
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757020
stem-surface            1980858
stem-color                   38
veil-type               2957489
veil-color              2740944
has-ring                     24
ring-type                128880
spore-print-color       2849678
habitat                      45
season                        0
dtype: int64

In [43]:
# Define the columns to check for missing values
columns_to_check = ['cap-shape', 'cap-color', 'does-bruise-or-bleed',
                    'gill-color', 'stem-color', 'has-ring', 'habitat']

# Drop rows with missing values in the specified columns
data = data.dropna(subset=columns_to_check)
data.isna().sum()

class                         0
cap-diameter                  0
cap-shape                     0
cap-surface              670948
cap-color                     0
does-bruise-or-bleed          0
gill-attachment          523876
gill-spacing            1258317
gill-color                    0
stem-height                   0
stem-width                    0
stem-root               2756822
stem-surface            1980676
stem-color                    0
veil-type               2957273
veil-color              2740741
has-ring                      0
ring-type                128850
spore-print-color       2849483
habitat                       0
season                        0
dtype: int64

In [44]:
# So many missing values, test data also has many missing values
# therefore I will replace missing values with 'Missing'
data = data.fillna('Missing')
data.isnull().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64

In [45]:
# Define the mapping dictionary
class_mapping = {'e': 0, 'p': 1}

# Map the 'class' column, handling any unmapped values
data['class'] = data['class'].apply(lambda x: class_mapping.get(x))
data

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0.0,8.80,f,s,u,f,a,c,w,4.51,...,Missing,Missing,w,Missing,Missing,f,f,Missing,d,a
1,1.0,4.51,x,h,o,f,a,c,n,4.79,...,Missing,y,o,Missing,Missing,t,z,Missing,d,w
2,0.0,6.94,f,s,b,f,x,c,w,6.85,...,Missing,s,n,Missing,Missing,f,f,Missing,l,w
3,0.0,3.88,f,y,g,f,s,Missing,g,4.16,...,Missing,Missing,w,Missing,Missing,f,f,Missing,d,u
4,0.0,5.85,x,l,w,f,d,Missing,w,3.37,...,Missing,Missing,w,Missing,Missing,f,f,Missing,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,0.0,9.29,f,Missing,n,t,Missing,Missing,w,12.14,...,b,Missing,w,u,w,t,g,Missing,d,u
3116941,0.0,10.88,s,Missing,w,t,d,c,p,6.65,...,Missing,Missing,w,Missing,Missing,f,f,Missing,d,u
3116942,1.0,7.82,x,e,e,f,a,Missing,w,9.51,...,Missing,Missing,y,Missing,w,t,z,Missing,d,a
3116943,0.0,9.45,p,i,n,t,e,Missing,p,9.13,...,Missing,y,w,Missing,Missing,t,p,Missing,d,u


In [46]:
# Function to calculate WoE and IV for categorical features
def calculate_woe_iv(df, feature, target):
    lst = []
    for val in df[feature].unique():
        n_events = df[(df[feature] == val) & (df[target] == 1)].shape[0]
        n_non_events = df[(df[feature] == val) & (df[target] == 0)].shape[0]
        n_total_events = df[df[target] == 1].shape[0]
        n_total_non_events = df[df[target] == 0].shape[0]
        
        # Avoid division by zero
        if n_non_events == 0 or n_events == 0:
            woe = 0
        else:
            woe = np.log((n_non_events / n_total_non_events) / (n_events / n_total_events))
        
        lst.append((val, n_events, n_non_events, woe))
    
    woe_df = pd.DataFrame(lst, columns=[feature, 'n_events', 'n_non_events', 'woe'])
    return woe_df

# Apply WoE encoding to each categorical column and replace the original column with the WoE values
categorical_columns = data.select_dtypes(include=['object']).columns

for column in categorical_columns:
    # Calculate WoE for the column
    woe_df = calculate_woe_iv(data, column, 'class')
    # Convert to a dictionary to map the WoE values
    woe_dict = woe_df.set_index(column)['woe'].to_dict()
    # Replace the original column with the WoE values
    data[column] = data[column].map(woe_dict)

# Verify that the columns are now numerical
print(data.dtypes)
print(data.head())

class                   float64
cap-diameter            float64
cap-shape               float64
cap-surface             float64
cap-color               float64
does-bruise-or-bleed    float64
gill-attachment         float64
gill-spacing            float64
gill-color              float64
stem-height             float64
stem-width              float64
stem-root               float64
stem-surface            float64
stem-color              float64
veil-type               float64
veil-color              float64
has-ring                float64
ring-type               float64
spore-print-color       float64
habitat                 float64
season                  float64
dtype: object
   class  cap-diameter  cap-shape  cap-surface  cap-color  \
0    0.0          8.80   0.141307     0.724180   0.031502   
1    1.0          4.51   0.148423     0.308800  -0.741724   
2    0.0          6.94   0.141307     0.724180   2.125481   
3    0.0          3.88   0.141307     0.341507   0.545164   
4    0.0 

In [47]:
data.dtypes

class                   float64
cap-diameter            float64
cap-shape               float64
cap-surface             float64
cap-color               float64
does-bruise-or-bleed    float64
gill-attachment         float64
gill-spacing            float64
gill-color              float64
stem-height             float64
stem-width              float64
stem-root               float64
stem-surface            float64
stem-color              float64
veil-type               float64
veil-color              float64
has-ring                float64
ring-type               float64
spore-print-color       float64
habitat                 float64
season                  float64
dtype: object

In [48]:
data.isnull().sum()

class                   4
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64

In [60]:
data=data.dropna()
data.isnull().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64

In [61]:
data['season'].unique()

array([-0.09921765,  0.82573645, -0.14378514,  0.64366213])

In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X = scaler.fit_transform(data.drop("class", axis=1))
y = data["class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [64]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0)
model = GBC.fit(X_train, y_train)
GBC.score(X_test, y_test)

0.8619975487050489

In [1]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))